In [ ]:
from chromatic_fitting import *
from chromatic import Rainbow, SimulatedRainbow
from pymc3 import Normal, Uniform, Model, HalfNormal,Deterministic,  plot_trace, sample_prior_predictive, sample_posterior_predictive
from tqdm import tqdm
from pymc3_ext import eval_in_model, optimize, sample, ParameterGroup
from exoplanet import QuadLimbDark, ImpactParameter

plt.matplotlib.style.use('default')

### Let's experiment with `+-*/`

In [ ]:
r = (SimulatedRainbow(R=5, dt=10*u.minute)
     .inject_transit()
     .inject_noise(signal_to_noise=1000))
r.plot()

Let's create two models and merge them using math (which is actually secretly creating `CombinedModel`s in the background.

In [ ]:
t = TransitModel()
t.setup_parameters(radius_ratio=WavelikeFitted(Normal, mu=0.3, sigma=0.05))

p = PolynomialModel()
p.setup_parameters(p=WavelikeFitted(Uniform,upper=[0.01,0.1],lower=[-0.01,-0.1],shape=2))

In [ ]:
c = t + p
c.attach_data(r)
c.setup_lightcurves()
c.setup_likelihood()
c.plot_priors()

In [ ]:
c = t - p
c.attach_data(r)
c.setup_lightcurves()
c.setup_likelihood()
c.plot_priors()

In [ ]:
c = t * p
c.attach_data(r)
c.setup_lightcurves()
c.setup_likelihood()
c.plot_priors()

In [ ]:
c = t / p
c.attach_data(r)
c.setup_lightcurves()
c.setup_likelihood()
c.plot_priors()

Can we actually do a fit?

In [ ]:
c = t + p
c.attach_data(r)
c.setup_lightcurves()
c.setup_likelihood()
c.optimize()
c.sample()

In [ ]:
c.plot_posteriors()

## Can we combine three models? 

(looks like not quite yet, but here's what maybe it would hopefully look like?)

In [ ]:
t2 = TransitModel()
t2.setup_parameters(epoch=0.05, radius_ratio=0.2, baseline=0)

In [ ]:
c = (t + t2) * p
c.attach_data(r)
c.setup_lightcurves()
c.setup_likelihood()
c.plot_priors()